<br>

<div align=center><font color=maroon size=6><b>TensorFlow 2 quickstart for beginners</b></font></div>

<br>

<font size=4><b>References:</b></font>
1. TF2 official tutorials: <a href="https://www.tensorflow.org/tutorials" style="text-decoration:none;">TensorFlow Tutorials</a> 
    * `TensorFlow > Learn > TensorFlow Core > `Tutorials > <a href="https://www.tensorflow.org/tutorials/quickstart/beginner" style="text-decoration:none;">TensorFlow 2 quickstart for beginners</a>
        * <a href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb" style="text-decoration:none;">Google Colab</a>

<br>
<br>
<br>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Load a prebuilt dataset.
1. Build a neural network machine learning model that classifies images.
2. Train this neural network.
3. Evaluate the accuracy of the model.

<br>
<br>
<br>

## Set up TensorFlow

In [1]:
import tensorflow as tf

In [2]:
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.5.0


<br>

If you are following along in your own development environment, rather than [Colab](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb), see the [install guide](https://www.tensorflow.org/install) for setting up TensorFlow for development.

<font color=maroon>**Note:**</font> Make sure you have upgraded to the latest `pip` to install the TensorFlow 2 package if you are using your own development environment. See the [install guide](https://www.tensorflow.org/install) for details.

<br>
<br>
<br>

## Load a dataset

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the sample data from integers to floating-point numbers:

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0

# 默认下载路径：C:\Users\18617\.keras\datasets

<br>
<br>
<br>

## Build a machine learning model

Build a `tf.keras.Sequential` model by stacking layers.

In [4]:
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape=(28, 28)),
                                    tf.keras.layers.Dense(128, activation='relu'),
                                    tf.keras.layers.Dropout(0.2),
                                    tf.keras.layers.Dense(10)])

<br>
<br>

For each example, the model returns a vector of [logits](https://developers.google.com/machine-learning/glossary#logits) or [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) scores, one for each class.

In [5]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.06038679,  0.41560248,  0.14996882, -0.6528467 , -0.10210094,
         0.0410786 ,  0.1958196 ,  0.7317687 ,  0.48372242,  0.52104795]],
      dtype=float32)

<br>
<br>

The `tf.nn.softmax` function converts these logits to *probabilities* for each class: 

In [6]:
tf.nn.softmax(predictions).numpy()

array([[0.08295103, 0.11832867, 0.09072495, 0.04065074, 0.07051057,
        0.08136476, 0.0949816 , 0.16233017, 0.12667009, 0.13148747]],
      dtype=float32)

In [7]:
tf.nn.softmax(predictions).numpy().sum()

1.0

<br>

<div class="alert alert-block alert-info">

<font size=3 color=red><b>NOTE: </b></font>
<br>
<br>
<font color=black size=3>It is possible to bake the `tf.nn.softmax` function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output. </font>

</div>

<br>
<br>

Define a loss function for training using `losses.SparseCategoricalCrossentropy`, which takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [8]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

<br>
<br>

This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [10]:
loss_fn(y_train[:1], predictions).numpy()

2.5088131

<br>
<br>

Before you start training, configure and compile the model using Keras `Model.compile`. Set the [`optimizer`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) class to `adam`, set the `loss` to the `loss_fn` function you defined earlier, and specify a metric to be evaluated for the model by setting the `metrics` parameter to `accuracy`.

In [11]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

<br>
<br>
<br>

## Train and evaluate your model

Use the `Model.fit` method to adjust your model parameters and minimize the loss: 

In [12]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2935 - accuracy: 0.9144
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1401 - accuracy: 0.9585
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1085 - accuracy: 0.9668
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0859 - accuracy: 0.9735
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0744 - accuracy: 0.9762


<br>
<br>

The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [13]:
model.evaluate(x_test, y_test, verbose=2)

313/313 - 1s - loss: 0.0727 - accuracy: 0.9779


[0.07267674803733826, 0.9779000282287598]

<br>

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

<br>
<br>

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [16]:
model_probability = tf.keras.Sequential([model, tf.keras.layers.Softmax()])

In [17]:
model_probability(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[4.55493546e-06, 5.89618345e-08, 2.95675018e-05, 6.62466628e-05,
        1.37343234e-10, 4.05931905e-06, 1.19042796e-11, 9.99841809e-01,
        1.96052351e-05, 3.39965954e-05],
       [2.89908808e-09, 9.78468961e-06, 9.99987721e-01, 8.82453037e-07,
        2.41824942e-16, 1.26304349e-06, 8.23135338e-09, 3.25673863e-11,
        3.98114423e-07, 3.95345378e-14],
       [3.26821976e-07, 9.99556720e-01, 1.02260776e-04, 5.30135731e-06,
        1.02491931e-05, 3.89768957e-06, 5.44201248e-06, 2.43478906e-04,
        7.19029413e-05, 3.43512085e-07],
       [9.98003185e-01, 7.38991091e-09, 8.74443038e-04, 5.55540396e-08,
        1.04438959e-05, 5.16843538e-05, 5.42363035e-04, 2.04532189e-04,
        7.12313022e-06, 3.06073809e-04],
       [1.17016327e-06, 4.03805416e-08, 1.72690306e-05, 2.35153280e-07,
        9.98623729e-01, 5.55158181e-07, 1.56505503e-05, 3.09988500e-05,
        5.47515810e-07, 1.30983815e-03]], dtype=float32)>

<br>
<br>
<br>

## Conclusion

Congratulations! You have trained a machine learning model using a prebuilt dataset using the [Keras](https://www.tensorflow.org/guide/keras/overview) API.

For more examples of using Keras, check out the [tutorials](https://www.tensorflow.org/tutorials/keras/). To learn more about building models with Keras, read the [guides](https://www.tensorflow.org/guide/keras). If you want learn more about loading and preparing data, see the tutorials on [image data loading](https://www.tensorflow.org/tutorials/load_data/images) or [CSV data loading](https://www.tensorflow.org/tutorials/load_data/csv).


<br>
<br>
<br>